In [1]:
library(tidyverse)

# Custom package
library(rutils)

-- Attaching packages ------------------------------------------------------------------------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.3     v purrr   0.3.4
v tibble  3.0.6     v dplyr   1.0.4
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1

-- Conflicts ---------------------------------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



In [2]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")

In [3]:
dset_idx <- 2

In [4]:
p_thresh <- 0.05
q_thresh <- 0.15

In [5]:
univ_surv_df <- read_tsv(paste0(dirs$analysis_dir, "/survival/", unified_dsets[dset_idx], "_univ_survival_results.tsv"))


-- Column specification ------------------------------------------------------------------------------------------------------------------------
cols(
  geneID = col_character(),
  km_pval = col_double(),
  km_qval = col_double(),
  cph_pval = col_double(),
  cph_qval = col_double(),
  coeff = col_double()
)




# Univariate Survival

In [6]:
sig_univ_surv_df <- univ_surv_df %>%
    dplyr::filter(km_pval < p_thresh | cph_pval < p_thresh) %>%
    dplyr::filter(km_qval < q_thresh | cph_qval < q_thresh)

In [7]:
nrow(sig_univ_surv_df)

[1] 1

In [8]:
sig_univ_surv_df

geneID,km_pval,km_qval,cph_pval,cph_qval,coeff
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
COL5A3,0.009416507,0.3484108,0.003190413,0.1074039,0.6365094


# Cox PH Lasso